In [ ]:
from traffic_simulator.structure import *
from time import sleep
from IPython.display import display
import ipywidgets as widgets
from streamz.dataframe import PeriodicDataFrame
from threading import Thread
from ipywidgets import interact
import pandas as pd

In [ ]:
get_new_cars = lambda: randint(0, 5)
get_size = lambda: randint(10, 50)
get_capacities = lambda: (randint(1, 5), randint(1, 5))
get_velocity = lambda: randint(3, 10)

city = City(3, 5, 0.8, 2, 5, get_new_cars, get_size, get_capacities, get_velocity)
city.draw()
print(city)

In [ ]:
for i in range(1):
    result = city.run()
    print(city)

# Definir database

In [ ]:
import sqlite3

#mydb = mysql.connector.connect(
#    host="localhost",
#    user="localuser",
#    password="Password_0",
#    auth_plugin="mysql_native_password",
#    port=3306
#)
mydb = sqlite3.connect('traffic_simulator.db', check_same_thread=False)

mycursor = mydb.cursor()

#mycursor.execute("DROP DATABASE traffic_simulator;")
#mycursor.execute("CREATE DATABASE traffic_simulator;")
#mycursor.execute("USE traffic_simulator;")

mycursor.execute("DROP TABLE cities;")
mycursor.execute("DROP TABLE cars;")
mycursor.execute("DROP TABLE etl_global;")
mycursor.execute("DROP TABLE etl_cities;")

mycursor.execute("CREATE TABLE cities (id CHAR(36) PRIMARY KEY, steps INT, n_crossing INT, n_streets INT, n_cars INT);")
mycursor.execute("CREATE TABLE cars (city CHAR(36), id INT, on_street INT, velocity INT, odometer INT);")

mycursor.execute("CREATE TABLE etl_global (total INT, n_crossing INT, n_streets INT, n_cars INT, avg_velocity INT, max_odometer INT, avg_steps FLOAT, max_n_cars INT, city_max_n_cars CHAR(36));")
mycursor.execute("CREATE TABLE etl_cities (id CHAR(36) PRIMARY KEY, n_crossing INT, n_streets INT, n_cars INT, avg_velocity INT, max_odometer INT, steps INT, avg_n_cars_streets INT);")

# Popular database

In [ ]:
city_index, steps, n_crossing, n_streets, n_cars, car_results = result

mycursor.execute(f"INSERT INTO cities (id, steps, n_crossing, n_streets, n_cars) VALUES ('{city_index}', {steps}, {n_crossing}, {n_streets}, {n_cars});")

for index, on_street, velocity, odometer in car_results:
    sql = f"INSERT INTO cars (city, id, on_street, velocity, odometer) VALUES ('{city_index}', {index}, {on_street}, {velocity}, {odometer});"
    mycursor.execute(sql)

# Popular ETL

In [ ]:
mycursor.execute("SELECT COUNT(id) FROM cities;")

total = int(mycursor.fetchone()[0])

mycursor.execute("SELECT SUM(n_crossing) FROM cities;")

n_crossing = int(mycursor.fetchone()[0])

mycursor.execute("SELECT SUM(n_streets) FROM cities;")

n_streets = int(mycursor.fetchone()[0])

mycursor.execute("SELECT SUM(n_cars) FROM cities;")

n_cars = int(mycursor.fetchone()[0])

mycursor.execute("SELECT AVG(velocity) FROM cars;")

avg_velocity = int(mycursor.fetchone()[0])

mycursor.execute("SELECT MAX(odometer) FROM cars;")

max_odometer = int(mycursor.fetchone()[0])

mycursor.execute("SELECT AVG(steps) FROM cities;")

avg_steps = int(mycursor.fetchone()[0])

mycursor.execute("SELECT MAX(n_cars) FROM cities;")

max_n_cars = int(mycursor.fetchone()[0])

mycursor.execute(f"SELECT id FROM cities WHERE n_cars = {max_n_cars};")

city_max_n_cars = str(mycursor.fetchone()[0])

mycursor.execute(f"INSERT INTO etl_global (total, n_crossing, n_streets, n_cars, avg_velocity, max_odometer, avg_steps, max_n_cars, city_max_n_cars) VALUES ({total}, {n_crossing}, {n_streets}, {n_cars}, {avg_velocity}, {max_odometer}, {avg_steps}, {max_n_cars}, '{city_max_n_cars}');")

mycursor.execute(f"SELECT * FROM cities;")

results = mycursor.fetchall()

for index, steps, n_crossing, n_streets, n_cars in results:
    index, steps, n_crossing, n_streets, n_cars = str(index), int(steps), int(n_crossing), int(n_streets), int(n_cars)
    
    mycursor.execute(f"SELECT AVG(velocity) FROM cars WHERE city = '{index}';")

    avg_velocity = int(mycursor.fetchone()[0])

    mycursor.execute(f"SELECT MAX(odometer) FROM cars WHERE city = '{index}';")

    max_odometer = int(mycursor.fetchone()[0])
    
    avg_n_cars_streets = n_cars / n_streets
    
    mycursor.execute(f"INSERT INTO etl_cities (id, n_crossing, n_streets, n_cars, avg_velocity, max_odometer, steps, avg_n_cars_streets) VALUES ('{index}', {n_crossing}, {n_streets}, {n_cars}, {avg_velocity}, {max_odometer}, {steps}, {avg_n_cars_streets});")

In [ ]:
delay = 0.5
report_type = 'Global'

report = pd.DataFrame()
view = PeriodicDataFrame(lambda **kwargs: report, interval=delay)

mycursor.execute(f"SELECT id FROM cities;")

cities = [city[0] for city in mycursor.fetchall()]

widget = widgets.Dropdown(
    options=['Global'] + cities,
    value='Global',
    description='View'
)

def on_change(change):
    global report_type
    
    if change['type'] == 'change' and change['name'] == 'value':
        report_type = change['new']

widget.observe(on_change)
def display_widget():
    display(widget)
    display(view)

In [ ]:
def get_report():
    global report, report_type, mydb
    
    last_report_type = ''
    
    while True:
        if report_type == 'Global':
            report = pd.read_sql_query("SELECT * FROM etl_global", mydb)
        else:
            report = pd.read_sql_query(f"SELECT * FROM etl_cities WHERE id = '{report_type}'", mydb)

        last_report_type = report_type

        sleep(delay)

In [ ]:
Thread(target = get_report).start()
display_widget()